## Find Start/Stop times of GC data 
- For stores matching specific requests in order to check correctness of new data

In [ ]:
import xarray as xr
import pandas as pd
import qgrid

#local
from utilities import search_df, add_time_info

In [ ]:
#dESGF = pd.read_csv('dESGF-CEDA.csv')  # from EvaluateResponse.ipynb, list of available datasets matching the request
#exps = dESGF.experiment_id.unique()
#variables = dESGF.variable_id.unique()  #[0]
#tables = dESGF.table_id.unique()

tables = 'Amon'
exps = ['historical']
variables = ['pr']
sources = 'EC-Earth3'

#d2 = pd.read_csv('https://cmip6.storage.googleapis.com/cmip6-zarr-consolidated-stores.csv')
#dGC = search_df(d2,variable_id=variables,experiment_id=exps,table_id=tables,source_id=sources)

In [ ]:
dfcat = pd.read_csv('https://cmip6.storage.googleapis.com/cmip6-zarr-consolidated-stores-noQC.csv')
widget=qgrid.show_grid(dfcat)#.drop(['institution_id','dcpp_init_year'],1))
print('GCS Catalog')
widget 

In [ ]:
# save results
dGC = widget.get_changed_df()     # use this to save filtered dataframe
dGC.variable_id.values

In [ ]:
dGCp = add_time_info(dGC,verbose=True)

In [ ]:
widget=qgrid.show_grid(dGCp)
widget

In [ ]:
dbad = widget.get_changed_df()     # use this to save filtered dataframe
dbad.values[0]

In [ ]:
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

exps = dGCp.experiment_id.unique()
for exp in exps:
    dGCpe = dGCp[dGCp.experiment_id==exp]
# use this to get number of models (one per model)
    dm = dGCpe[['experiment_id','source_id','member_id','variable_id','start','stop','nt']].groupby([
                'experiment_id','member_id','source_id','variable_id','start','stop','nt']).nunique()[['member_id']]

    table = pd.DataFrame.pivot_table(dm,
                                     values='member_id',
                                     index=['source_id','variable_id','start','stop','nt'],
                                     columns=[],
                                     aggfunc=np.sum,
                                     fill_value=0)
    print(exp,'\n',table)

In [ ]:
ds = xr.open_zarr(fsspec.get_mapper('gs://cmip6/ScenarioMIP/NCC/NorESM2-LM/ssp126/r1i1p1f1/day/tasmax/gn/'),consolidated=True)

In [ ]:
http://noresg.nird.sigma2.no/thredds/fileServer/esg_dataroot/cmor/CMIP6/ScenarioMIP/NCC/NorESM2-LM/ssp126/r1i1p1f1/day/pr/gn/v20191108/pr_day_NorESM2-LM_ssp126_r1i1p1f1_gn_20310101-20401230.nc
http://noresg.nird.sigma2.no/thredds/fileServer/esg_dataroot/cmor/CMIP6/ScenarioMIP/NCC/NorESM2-LM/ssp126/r1i1p1f1/day/pr/gn/v20191108/pr_day_NorESM2-LM_ssp126_r1i1p1f1_gn_20310101-20401231.nc

In [ ]:
ds2 = xr.open_zarr(fsspec.get_mapper('gs://cmip6/ScenarioMIP/NCC/NorESM2-LM/ssp245/r1i1p1f1/day/tasmax/gn/'),consolidated=True)